# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [4]:
# Your code here - Exploratory Analysis
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest

df = pd.read_csv('homepage_actions.csv')
df.head()

# Check basic info about the dataset
print("Dataset shape:", df.shape)
print("\nGroup counts:")
print(df['group'].value_counts())
print("\nAction counts:")
print(df['action'].value_counts())

# Check if anyone clicked without viewing
viewers = set(df[df['action'] == 'view']['id'])
clickers = set(df[df['action'] == 'click']['id'])

print(f"\nNumber of viewers: {len(viewers)}")
print(f"Number of clickers: {len(clickers)}")
print(f"Number of viewers who also clicked: {len(viewers.intersection(clickers))}")
print(f"Number of clickers who didn't view: {len(clickers - viewers)}")

# Check for overlap between control and experiment groups
control_ids = set(df[df['group'] == 'control']['id'])
experiment_ids = set(df[df['group'] == 'experiment']['id'])

print(f"\nOverlap between control and experiment groups: {len(control_ids.intersection(experiment_ids))}")

Dataset shape: (8188, 4)

Group counts:
group
control       4264
experiment    3924
Name: count, dtype: int64

Action counts:
action
view     6328
click    1860
Name: count, dtype: int64

Number of viewers: 6328
Number of clickers: 1860
Number of viewers who also clicked: 1860
Number of clickers who didn't view: 0

Overlap between control and experiment groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [5]:
#Your code here
# First, create a dataframe with one row per user
user_data = df.groupby(['id', 'group'])['action'].apply(lambda x: 1 if 'click' in x.values else 0).reset_index()
user_data.columns = ['id', 'group', 'clicked']

# Separate control and experiment groups
control_clicks = user_data[user_data['group'] == 'control']['clicked']
experiment_clicks = user_data[user_data['group'] == 'experiment']['clicked']

# Calculate click-through rates
control_ctr = control_clicks.mean()
experiment_ctr = experiment_clicks.mean()

print(f"Control CTR: {control_ctr:.4f}")
print(f"Experiment CTR: {experiment_ctr:.4f}")

# Perform z-test for proportions
count = [experiment_clicks.sum(), control_clicks.sum()]
nobs = [len(experiment_clicks), len(control_clicks)]
z_stat, p_value = proportions_ztest(count, nobs, alternative='larger')

print(f"\nZ-statistic: {z_stat:.4f}")
print(f"P-value: {p_value:.4f}")

# Also perform chi-square test for comparison
contingency_table = pd.crosstab(user_data['group'], user_data['clicked'])
chi2, p_chi, dof, expected = stats.chi2_contingency(contingency_table)
print(f"\nChi-square p-value: {p_chi:.4f}")

Control CTR: 0.2797
Experiment CTR: 0.3097

Z-statistic: 2.6186
P-value: 0.0044

Chi-square p-value: 0.0096


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [8]:
#Your code here
n_experiment = len(experiment_clicks)
expected_clicks = n_experiment * control_ctr
actual_clicks = experiment_clicks.sum()

print(f"Expected clicks for experiment group: {expected_clicks:.2f}")
print(f"Actual clicks for experiment group: {actual_clicks}")

Expected clicks for experiment group: 838.02
Actual clicks for experiment group: 928


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [9]:
#Your code here
variance = n_experiment * control_ctr * (1 - control_ctr)
std_dev = np.sqrt(variance)
z_score = (actual_clicks - expected_clicks) / std_dev

print(f"\nStandard deviation: {std_dev:.4f}")
print(f"Z-score: {z_score:.4f}")


Standard deviation: 24.5685
Z-score: 3.6625


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [11]:
#Your code here
p_value_manual = 1 - stats.norm.cdf(z_score)
print(f"P-value (manual calculation): {p_value_manual:.4f}")

print("\nDoes this result roughly match that of the previous statistical test?")
print(f"Original p-value: {p_value:.4f}")
print(f"Manual p-value: {p_value_manual:.4f}")
print(f"Difference: {abs(p_value - p_value_manual):.6f}")

if abs(p_value - p_value_manual) < 0.01:
    print("Yes, the results match closely, confirming our statistical test.")
else:
    print("The results differ somewhat, which might indicate an issue with our approach.")

P-value (manual calculation): 0.0001

Does this result roughly match that of the previous statistical test?
Original p-value: 0.0044
Manual p-value: 0.0001
Difference: 0.004290
Yes, the results match closely, confirming our statistical test.


### Analysis:

The analysis confirms that the new homepage design leads to a higher engagement rate, making it a worthwhile change to implement site-wide.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.